In [2]:
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, RDFS
from rdflib import Graph

#14	Nintendo shop - https://store.nintendo.nl/nl
#Zijian Dong -i6226172	Anqi Fan -i6272224	Qi Cui -i6288444	Tianyu Wei -i6231911	Zhefan Cheng -i6267501


In [3]:
# Make a graph
g = Graph()
NS = Namespace('https://store.nintendo.nl/nl/')
schema = Namespace("http://schema.org/")
nintendo = Literal("Nintendo")
#--------------Categories------------------------------------

console = NS['consoles']
games = NS['games']
merchandise = NS['merchandise'] #周边merchandise
franchise = NS['franchises'] #独家ip 和其它的会重复Exclusive IP and others will be duplicated
new_product = NS['new']

#--------Sub-Categories--------------------------------------
#--------Sub Categories for consoles-------------------------
switch = NS['consoles/nintendo-switch-consoles'] #是console的product
switch_bundle = NS['consoles/nintendo-switch-bundles'] #是console的product
accessory = NS['consoles/accessories'] #是switch的配件
#-------Sub Categories for merchandise-----------------
apparel = NS['merchandise/apparel']
home_and_gift = NS['merchandise/home-and-gifts']


In [4]:
g.bind('NS', NS)

#--------Predicates--------------------------------------------
product = schema['Product']
isAccessoryOf = schema['isAccessoryOrSparePartFor']
isVariantOf = schema['isVariantOf']
videoGame = schema['videoGame']
#--------Instance--------------------------------------

#Define types for Mainpage
g.add((console, RDF.type, nintendo)) # add的三个参数："_SubjectType", "_PredicateType", "_ObjectType"
g.add((games, RDF.type, nintendo))
g.add((merchandise, RDF.type, nintendo))
g.add((franchise, RDF.type, nintendo))
g.add((new_product, RDF.type, nintendo))

#Define subclasses of main categories
g.add((switch, RDFS.subClassOf, console))
g.add((switch_bundle, RDFS.subClassOf, console))
g.add((accessory, isAccessoryOf, console))

g.add((apparel, RDFS.subClassOf, merchandise))
g.add((home_and_gift, RDFS.subClassOf, merchandise))


<Graph identifier=N014f01e75c0346c78c58c86f6249bcb8 (<class 'rdflib.graph.Graph'>)>

In [ ]:
import pandas as pd

df = pd.read_excel("new2.xlsx")

for i,row in df.iterrows():
    # 判断URI是否为空 跳过没有爬到的数据
    if not pd.isna(row["Item_URI"]) :
        product = NS[row["Item_URL"].replace("https://store.nintendo.nl/nl/","").replace(" ","")]
        g.add((product, RDF.type, schema.Product))
        g.add((product, schema.name, Literal(row["Value_1"])))
        g.add((product, schema.description, Literal(row["Value_2"])))
        g.add((product, schema.sku, Literal(row["Value_3"])))
        g.add((product, schema.price, Literal(row["Value_4"])))
        g.add((product, schema.brand, Literal("Nintendo"))) 
    #    graph.add((product, schema[row["link"]],eval(row["type"])))
